In [ ]:
pip install --upgrade transformers torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 46.0 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.2
    Uninstalling transformers-4.46.2:
      Successfully uninstalled transformers-4.46.2


In [ ]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("Clinical-AI-Apollo/Medical-NER")
model = AutoModelForTokenClassification.from_pretrained("Clinical-AI-Apollo/Medical-NER")

tokenizer_config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.66M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/5.14k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/736M [00:00<?, ?B/s]

In [ ]:
# Perform NER on patients' clinical notes. Extract the disease, age, and sex of
# each patient in the dataset based on physicians' clinical notes.

import pandas as pd

pipe = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple") # pass device=0 if using gpu

df = pd.read_csv('/content/patient_data.csv')
df = df.dropna(how='all')

patient_dict = {}

for row in range(len(df)):
  text = df.iloc[row]['clinical_note']
  if text is None:
    continue
  results = pipe(text)
  population = {}
  for result in results:
    for key in result:
      if result[key] == 'DISEASE_DISORDER' or result[key] == 'AGE' or result[key] == 'SEX':
        population[result[key]] = result['word']
  patient_dict[row] = population

print(patient_dict)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{0: {'DISEASE_DISORDER': 'tuberculosis', 'AGE': 'up'}, 1: {}, 2: {'AGE': '62-year-old', 'SEX': 'male', 'DISEASE_DISORDER': 'TB'}, 3: {'SEX': 'Female', 'AGE': '25', 'DISEASE_DISORDER': 'tuberculosis'}, 4: {'SEX': 'Male', 'AGE': '50', 'DISEASE_DISORDER': 'tuberculosis'}, 5: {'DISEASE_DISORDER': 'TB'}, 6: {'DISEASE_DISORDER': 'syphilis', 'AGE': 'up'}, 7: {'AGE': 'up', 'DISEASE_DISORDER': 'syphilis'}, 8: {'DISEASE_DISORDER': 'hemolytic complications'}, 9: {'AGE': '35-year-old', 'SEX': 'male', 'DISEASE_DISORDER': 'smallpox'}, 10: {'AGE': '29-year-old', 'SEX': 'female', 'DISEASE_DISORDER': 'herpes simplex'}, 11: {'AGE': '42-year-old', 'SEX': 'male', 'DISEASE_DISORDER': 'organ dysfunction'}, 12: {'DISEASE_DISORDER': 'tularemia'}, 13: {'AGE': '50-year-old', 'SEX': 'male', 'DISEASE_DISORDER': 'tuberculoma'}, 14: {'AGE': '34-year-old', 'SEX': 'female', 'DISEASE_DISORDER': 'tuberculosis'}, 15: {'DISEASE_DISORDER': 'mastoid tuberculosis'}, 16: {'AGE': '40-year-old', 'DISEASE_DISORDER': 'pneumocyst

In [ ]:
# Cache as CSV for later use
df = pd.DataFrame.from_dict(patient_dict, orient='index')

# Save to CSV
df.to_csv('patient_population_data.csv')

patient_df = pd.read_csv('patient_population_data.csv')
patient_df = patient_df.dropna(subset=['DISEASE_DISORDER'])

print(len(patient_df))

198


In [ ]:
# Clean patient data

patient_df['AGE'] = patient_df['AGE'].str.extract('(\d+)', expand=False)
patient_df['AGE'] = patient_df['AGE'].astype('Int64')

patient_df['SEX'] = patient_df['SEX'].str.lower()

print(patient_df)

patient_df.drop(columns=['Unnamed: 0'], axis=1)
# df = pd.DataFrame.from_dict(trial_dict, orient='index')

# Save to CSV
patient_df.to_csv('patient_population_data.csv')

     Unnamed: 0 DISEASE_DISORDER   AGE     SEX
0             0     tuberculosis  <NA>     NaN
1             2               TB    62    male
2             3     tuberculosis    25  female
3             4     tuberculosis    50    male
4             5               TB  <NA>     NaN
..          ...              ...   ...     ...
193         225      degree burn    37    male
194         226     blast injury    55    male
195         227        parasitic  <NA>  female
196         228     hearing loss    48    male
197         229     hypertension    43    male

[198 rows x 4 columns]


In [ ]:
# Clean and make sense of clinical trial data
import pandas as pd

trial_df = pd.read_csv('trial_data.csv')
trial_dict = {} # Maps from trial ID to the population it treats

trial_df['Sex'] = trial_df['Sex'].fillna('').astype(str).str.lower()
trial_df['Age'] = trial_df['Age'].fillna('').astype(str).str.lower()

age_dict = {"child": (0, 18),
            "child, adult": (0, 65),
            "child, adult, older_adult": (0, 100),
            "adult": (18, 65),
            "adult, older_adult": (18, 100),
            "older_adult": (65, 100)}

sex_dict = {"male": 0,
            "female": 1}

for row in range(len(trial_df)):
  sex = 2

  if trial_df.iloc[row]['Sex'] in sex_dict:
    sex = sex_dict[trial_df.iloc[row]['Sex']]

  age = (0, 100)

  if trial_df.iloc[row]['Age'] in age_dict:
    age = age_dict[trial_df.iloc[row]['Age']]

  curr_trial = {"Condition": trial_df.iloc[row]['Conditions'],
                "Age": age,
                "Sex": sex}

  trial_dict[trial_df.iloc[row]['NCT Number']] = curr_trial

print(trial_dict)

<ipython-input-2-20d4bb47e277>:4: DtypeWarning: Columns (12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,110,111) have mixed types. Specify dtype option on import or set low_memory=False.
  trial_df = pd.read_csv('trial_data.csv')


{'NCT06706713': {'Condition': 'Non-Small Cell Lung Cancer|EGFR Exon 20 Insertion Mutation', 'Age': (18, 100), 'Sex': 2}, 'NCT06706700': {'Condition': 'Intraductal Papillary Mucinous Neoplasm of Pancreas|Pancreatic Cyst', 'Age': (18, 100), 'Sex': 2}, 'NCT06706687': {'Condition': 'Bipolar Disorder|Frontotemporal Dementia, Behavioral Variant', 'Age': (18, 100), 'Sex': 2}, 'NCT06706674': {'Condition': 'Irritability Associated With Autism Spectrum Disorder', 'Age': (0, 18), 'Sex': 2}, 'NCT06706661': {'Condition': 'Malnutrition', 'Age': (18, 100), 'Sex': 2}, 'NCT06706648': {'Condition': 'Occupational Hardiness|Work Engagement|Psychological Capital Interventions', 'Age': (18, 65), 'Sex': 2}, 'NCT06706635': {'Condition': 'Aortic Stenosis Disease', 'Age': (18, 100), 'Sex': 2}, 'NCT06706622': {'Condition': 'Multiple System Atrophy', 'Age': (18, 100), 'Sex': 2}, 'NCT06706609': {'Condition': 'Diabetes Mellitus|Diabetes Mellitus, Adult-Onset|Diabetes Mellitus Type 1 and 2', 'Age': (0, 100), 'Sex': 

In [ ]:
df = pd.DataFrame.from_dict(trial_dict, orient='index')

# Save to CSV
df.to_csv('trial_population_data.csv')